# NEU PCA
---

## Parameters

In [2]:
PCA_Rank = 4

#### NEW IMPORTS - TBD

In [3]:
from sklearn.decomposition import PCA
from sklearn import datasets

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import math
import datetime

In [4]:
# First Round Initializations (Global Level) #
#============================================#
# Load Dependances and makes path(s)
exec(open('Initializations_Dump.py').read())
# Load Hyper( and meta) parameter(s)
exec(open('HyperParameter_Grid.py').read())
# %run Helper_Functions.ipynb
exec(open('Helper_Functions.py').read())
# Load Models
# %run Architecture_Builder.ipynb
exec(open('Architecture_Builder.py').read())

Using TensorFlow backend.


Parameter Grids Build and Loaded!
Complete NEU-Structure Building Procedure!!!
Complete NEU-Structure Building Procedure!!!
Complete NEU-ffNN Training Procedure!!!
Deep Feature Builder - Ready
Complete NEU-ffNN Training Procedure!!!
Complete NEU-ffNN Training Procedure!!!
Complete NEU-ffNN (Fully Coupled) Training Procedure!!!
Complete NEU-ffNN Training Procedure!!!
Complete NEU-ffNN Training Procedure!!!
Complete NEU-PCA Training Procedure!!!


## Prepare Data

### Load Data

In [10]:
# Load Data
yield_data = pd.read_excel('inputs/data/ust_daily.ods', engine='odf')

### Format Data

In [11]:
yield_data['date'] = yield_data['date'].apply(lambda x: x.strftime('%m/%d/%Y'))
dates_index = yield_data['date']
yield_data.set_index('date', drop=True, inplace=True)
yield_data.index.names = [None]
Yield_data = yield_data#.transpose()

### Sub-set Data

In [32]:
# Get indices
N_train_step = int(round(yield_data.shape[0]*Train_step_proportion,0))
N_test_set = int(yield_data.shape[0] - round(yield_data.shape[0]*Train_step_proportion,0))
# Get Datasets
X_train = yield_data[:N_train_step]
X_test = yield_data[-N_test_set:]
# Update User
print('#================================================#')
print(' Training Datasize: '+str(X_train.shape[0])+' and test datasize: ' + str(X_test.shape[0]) + '.  ')
print('#================================================#')

#================================================#
 Training Datasize: 5126 and test datasize: 1708.  
#================================================#


### Preview Data

In [34]:
print('Training Dataset Preview:')
print(X_train)
print('Testing Dataset Preview:')
print(X_test)

Training Dataset Preview:
            BC_1MONTH  BC_3MONTH  BC_6MONTH  BC_1YEAR  BC_2YEAR  BC_3YEAR  \
06/08/1990       0.00       7.96       8.03      8.07      8.33      8.38   
06/07/1990       0.00       7.96       8.01      8.07      8.34      8.38   
06/11/1990       0.00       7.99       8.07      8.09      8.36      8.41   
06/13/1990       0.00       7.93       7.99      8.02      8.27      8.32   
06/12/1990       0.00       7.99       8.05      8.09      8.36      8.42   
...               ...        ...        ...       ...       ...       ...   
09/10/2010       0.10       0.14       0.19      0.27      0.58      0.88   
09/09/2010       0.10       0.14       0.19      0.26      0.57      0.87   
09/28/2010       0.08       0.16       0.20      0.26      0.37      0.64   
09/27/2010       0.08       0.16       0.19      0.26      0.44      0.66   
09/29/2010       0.12       0.16       0.20      0.27      0.44      0.67   

            BC_5YEAR  BC_7YEAR  BC_10YEAR  BC_20Y

---

### Fix Seeds for Reproducability

In [30]:
# Numpy
np.random.seed(2020)
# Tensorflow
tf.random.set_seed(2020)
# Python's Seed
random.seed(2020)

---

## Get Data-Driven Robustness Hyper-Parameters

Data Driven Robustness Rule:
Using [this article](https://www.sciencedirect.com/science/article/pii/S0378375815000403) for the variance estimate.  
$$
\hat{\sigma}^2 \triangleq
\frac1{(n-1)}\sum_{1<n\leq N} (y_n - y_{n-1})^2
.
$$

We couple it to the problem via the following
$$
\underset{\underset{0\leq w_n\leq 1}{\sum_{n\leq N} w_n=1}}{\operatorname{argmax}} \sum_{n\leq N} w_n L(f(x_n),\hat{f}(x_n),x_n) - \sigma^2 \sum_{n\leq N} w_n \ln\left(\frac{w_n}{N}\right)
.
$$

In [35]:
# GET STATISTICAL VARIANCE ESTIMATE
var_estimate = np.sum(np.diff(X_train)**2)/(((X_train.shape[0])-1))
var_estimate = np.float(round(var_estimate,3))


robustness_dictionary = {'robustness_parameter': [var_estimate*factor,var_estimate*2*factor,var_estimate*3*factor]}
#==================================================================================#        
### Create NEU parameter disctionary by parameters joining model it is upgrading ###
#==================================================================================#
param_grid_NEU_Nets = {**param_grid_NEU_Nets,
                       **robustness_dictionary}

param_grid_NEU_Feature_Only_Nets = {**param_grid_NEU_Feature_Only_Nets,
                                    **robustness_dictionary}

NEU_Structure_Dictionary = {**NEU_Structure_Dictionary,
                            **robustness_dictionary}

# NEU OLS Keys
param_grid_NEU_Reg_Nets = {**param_grid_NEU_Nets,
                           **robustness_dictionary}

param_grid_NEU_Reg_Nets.pop('height', None)
param_grid_NEU_Reg_Nets.pop('depth', None)
print(" ")

# Benchark(s)
---

## Perform PCA

In [36]:
# Compute/Initialize PCA
pca = PCA(n_components=PCA_Rank)
pca.fit(X_train)

# Define PCA Rreconstructor
def PCA_reconstructor(x_in):
    affine_parameterizer = np.matmul(pca.components_,(x_in-pca.mean_))
    low_dimensional_reconstructed = np.matmul(np.transpose(pca.components_), affine_parameterizer) + pca.mean_
    return low_dimensional_reconstructed

In [37]:
reconstructed_data = np.zeros(X_train.shape)
for i in range(X_train.shape[0]):
    reconstructed_data[i:]=PCA_reconstructor(X_train[i,])

# reconstruction_Errors
reconstruction_errors = np.mean(np.abs(reconstructed_data-X_train),axis=-1)
print(np.mean(reconstruction_errors))

KeyError: (0,)

---